# Modules

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime, date, time
import seaborn as sns
from scipy import stats
import scipy.stats as st
import glob

# Dataset

The dataset used in this analysis consists of beer reviews from two beer rating websites,**BeerAdvocate** and **RateBeer**, for a period ranging from 2001 to 2017. For each website, we have 5 files:
- users.csv: metadata about reviewers
- beers.csv : metadata about reviewed beers
- breweries.csv : metadata about breweries
- ratings.txt : all reviews given by users, including numerical ratings and sometimes textual reviews
- reviews.txt : only reviews given by users that include both numerical ratings and textual reviews

In our analysis, we will not use textual reviews. Thus, we will only use ratings.txt files and not reviews.txt files, as we will use all reviews, whether or not they include textual reviews.

### Load data into Dataframes

The .csv files are not too large and can efficiently be loaded into DataFrames.

In [40]:
BA_DATA_FOLDER = 'data/BeerAdvocate/'
RB_DATA_FOLDER = 'data/RateBeer/'

BA_USERS = BA_DATA_FOLDER+"users.csv"
BA_BEERS = BA_DATA_FOLDER+"beers.csv"
BA_BREWERIES = BA_DATA_FOLDER+"breweries.csv"

RB_USERS = RB_DATA_FOLDER+"users.csv"
RB_BEERS = RB_DATA_FOLDER+"beers.csv"
RB_BREWERIES = RB_DATA_FOLDER+"breweries.csv"

In [41]:
ba_users = pd.read_csv(BA_USERS)
ba_beers = pd.read_csv(BA_BEERS)
ba_breweries = pd.read_csv(BA_BREWERIES)

rb_users = pd.read_csv(RB_USERS)
rb_beers = pd.read_csv(RB_BEERS)
rb_breweries = pd.read_csv(RB_BREWERIES)

On the other hand, the ratings.txt files are extremely large, and trying to load them directly into DataFrames leads to kernel freezes. In order to circumvent this problem, we wrote a script (review_parser.py, located in src/scripts), which processes each rating file by dividing it into parts, parsing each part, and saving as JSON. In the notebook, we then load the different JSON files into DataFrames, that we concatenate. Dividing the large .txt files into smaller JSON chunks and then loading each chunk separately, avoids trying to load the entire file into memory at once, which can cause kernel freezes due to memory overload. In addition, JSON is a format that pandas can read efficiently.

In [5]:
# Load BeerAdvocate ratings stored in json files into a single DataFrame
ba_json_files = glob.glob(BA_DATA_FOLDER+'*.json')
ba_df_list = [pd.read_json(file) for file in ba_json_files]
ba_ratings = pd.concat(ba_df_list, ignore_index=True)
ba_ratings.head()

C:\Users\abdul\AppData\Local\Temp\ipykernel_24580\3299018877.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  ba_df_list = [pd.read_json(file) for file in ba_json_files]
C:\Users\abdul\AppData\Local\Temp\ipykernel_24580\3299018877.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  ba_df_list = [pd.read_json(file) for file in ba_json_files]


,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating
0,Régab,142544.0,Societe des Brasseries du Gabon (SOBRAGA),37262.0,Euro Pale Lager,4.5,2015-08-20 10:00:00,nmann08,nmann08.184925,3.25,2.75,3.25,2.75,3.0,2.88
1,Barelegs Brew,19590.0,Strangford Lough Brewing Company Ltd,10093.0,English Pale Ale,4.5,2009-02-20 11:00:00,StJamesGate,stjamesgate.163714,3.00,3.50,3.50,4.00,3.5,3.67
2,Barelegs Brew,19590.0,Strangford Lough Brewing Company Ltd,10093.0,English Pale Ale,4.5,2006-03-13 11:00:00,mdagnew,mdagnew.19527,4.00,3.50,3.50,4.00,3.5,3.73
3,Barelegs Brew,19590.0,Strangford Lough Brewing Company Ltd,10093.0,English Pale Ale,4.5,2004-12-01 11:00:00,helloloser12345,helloloser12345.10867,4.00,3.50,4.00,4.00,4.5,3.98
4,Barelegs Brew,19590.0,Strangford Lough Brewing Company Ltd,10093.0,English Pale Ale,4.5,2004-08-30 10:00:00,cypressbob,cypressbob.3708,4.00,4.00,4.00,4.00,4.0,4.00


In [6]:
# Load RateBeer ratings stored in json files into a single DataFrame
rb_json_files = glob.glob(RB_DATA_FOLDER+'*.json')
rb_df_list = [pd.read_json(file) for file in rb_json_files]
rb_ratings = pd.concat(rb_df_list, ignore_index=True)
rb_ratings.head()

C:\Users\abdul\AppData\Local\Temp\ipykernel_24580\180671459.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  rb_df_list = [pd.read_json(file) for file in rb_json_files]
C:\Users\abdul\AppData\Local\Temp\ipykernel_24580\180671459.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  rb_df_list = [pd.read_json(file) for file in rb_json_files]


,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating
0,33 Export (Gabon),410549.0,Sobraga,3198.0,Pale Lager,5.0,2016-04-26 10:00:00,Manslow,175852.0,2.0,4.0,2.0,4.0,8.0,2.0
1,Castel Beer (Gabon),105273.0,Sobraga,3198.0,Pale Lager,5.2,2017-02-17 11:00:00,MAGICuenca91,442761.0,2.0,3.0,2.0,4.0,8.0,1.9
2,Castel Beer (Gabon),105273.0,Sobraga,3198.0,Pale Lager,5.2,2016-06-24 10:00:00,Sibarh,288889.0,3.0,3.0,2.0,3.0,5.0,1.6
3,Castel Beer (Gabon),105273.0,Sobraga,3198.0,Pale Lager,5.2,2016-01-01 11:00:00,fombe89,250510.0,4.0,3.0,1.0,2.0,5.0,1.5
4,Castel Beer (Gabon),105273.0,Sobraga,3198.0,Pale Lager,5.2,2015-10-23 10:00:00,kevnic2008,122778.0,2.0,4.0,2.0,4.0,7.0,1.9


### First look at the data

We will now examine the different DataFrames in more detail.

In [42]:
# explain the columns of users, beers, breweries and ratings DataFrames

**BeerAdvocate beer Dataframe**

In [43]:
ba_beers.sample(4)

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,nbr_reviews,avg,ba_score,bros_score,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
13939,191061,Farmers Daughter,19739,Rockbottom Bar & Brewery,Belgian Pale Ale,0,0,NaN,NaN,NaN,6.0,NaN,NaN,0,NaN
101840,72285,Box Set Track #11 - The Devil Went Down To Geo...,18149,The Lost Abbey,American Barleywine,70,16,3.55,82.0,NaN,12.0,3.735,NaN,0,NaN
48523,189855,Ripa,41521,Devils Brewery,American IPA,1,1,4.41,NaN,NaN,5.0,4.410,NaN,0,NaN
54370,221012,Arequipa,44772,Trópica Brewing Co.,American IPA,1,1,4.22,NaN,NaN,4.8,4.220,NaN,0,NaN


Let us explain the different columns of the BeerAdvocate beer Dataframe, in which each row is a beer:
- beer_id, beer_name, brewery_id, brewery_name, style are explicit
- nbr_ratings: total number of reviews for that beer, whether or not they include textual reviews
- nbr_reviews: number of reviews for that beer that include textual reviews
- avg: average rating (out of 5) given to the beer based on user ratings
- ba_score: the BeerAdvocate score assigned to the beer, which corresponds to the beer's overall rating within its style category, calculated using a trimmed mean and a custom Bayesian formula that adjusts for the beer's style, balancing the score based on the number of ratings and the style's average
- bros_score: beer rating given by the site’s founders
- abv: 'Alcohol by volume', which indicates the percentage of alcohol content in the beer
- avg_computed: average rating (out of 5) recalculated using a weighted sum of the different aspect ratings
- zscore: z-score of the beer's average rating, which is a statistical measure that indicates how many standard deviations the average rating is from the mean of all ratings from the BeerAdvocate dataset
- nbr_matched_valid_ratings: number of valid ratings for beers that were successfully matched between two BeerAdvocate and RateBeer
- avg_matched_valid_ratings: average rating of those matched and valid ratings across the sites

The last two columns are related to the analysis performed by Robert West and Gael Lederrey in the following paper: https://dlab.epfl.ch/people/west/pub/Lederrey-West_WWW-18.pdf.

**RateBeer beer Dataframe**

In [44]:
rb_beers.sample(4)

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,overall_score,style_score,avg,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
332155,74947,Fort Street Deadhead Dunkelweizen,5691,Fort Street Brewery,Dunkelweizen,2,NaN,NaN,3.01,6.0,3.800000,NaN,0,NaN
29579,243027,Wellington Welly One-Off Cedar Rye PA,42,Wellington Brewery,Specialty Grain,1,NaN,NaN,2.84,5.7,3.400000,NaN,0,NaN
18941,337195,Four Winds Salt & Pepper Gose,16851,Four Winds Brewing,Grodziskie/Gose/Lichtenhainer,7,NaN,NaN,3.21,5.0,3.357143,-0.167193,7,3.357143
428954,47555,De Troch Kriekenlambik,363,Brouwerij De Troch,Lambic Style - Fruit,55,47.0,26.0,3.15,NaN,3.169091,NaN,0,NaN


Let us explain the different columns of the RateBeer beer Dataframe, in which each row is a beer:

The beer_id, beer_name, brewery_id, brewery_name, style, nbr_ratings, avg, abv, avg_computed, z-score, nbr_matched_valid_ratings and avg_matched_valid_ratings are the same as for the BeerAdvocate beer Dataframe.

Some columns are missing compared to the BeerAdvocate beer Dataframe: ba_score and bros_score (which makes sense as these are BeerAdvocate-specific scores), and nbr_reviews.

New columns are present compared to the BeerAdvocate beer Dataframe:
- overall_score: score (out of 100) which "reflects the rating given by RateBeer users and how this beer compares to all other beers on RateBeer", calculated by considering the ratings given by each user and the total number of ratings for the beer
- style_score: score given to the beer (out of 100) specifically within its style category

**BeerAdvocate user Dataframe**

In [45]:
ba_users.sample(4)

,nbr_ratings,nbr_reviews,user_id,user_name,joined,location
69258,2,0,onclouud9.968944,onclouud9,1.428055e+09,"United States, Virginia"
84576,8,0,romerojo.965370,Romerojo,1.427454e+09,NaN
89230,3,0,jsterrett0412.841561,JSterrett0412,1.407492e+09,"United States, Pennsylvania"
75202,26,0,taylorplaysmusic.666459,taylorplaysmusic,1.332155e+09,NaN


Let us explain the different columns of the BeerAdvocate user Dataframe, in which each row is a reviewer:
- nbr_ratings, nbr_reviews, user_id, user_name, and location are explicit
- joined: timestamp indicating when the user joined BeerAdvocate in Unix timestamp format (the number of seconds since January 1, 1970, 00:00:00 UTC)

**RateBeer user Dataframe**

In [46]:
rb_users.sample(4)

,nbr_ratings,user_id,user_name,joined,location
57384,1,3438,EdB,1.014030e+09,"United States, Utah"
12891,3,146926,coffygrinder,1.324465e+09,England
27703,1,477538,Marauzer,1.498471e+09,NaN
22908,1,63236,Gornography,1.193742e+09,NaN


Let us explain the different columns of the RateBeer user Dataframe, in which each row is a reviewer:

The columns are the same as in the BeerAdvocate user Dataframe (joined is obviously the timestamp indicating when the user joined RateBeer and not BeerAdvocate), except that nbr_reviews is missing.

**Brewery Dataframes**

In [47]:
ba_breweries.sample(4)

,id,location,name,nbr_beers
1882,36695,Canada,Siboire - Jacques-Cartier,1
11733,18566,"United States, Minnesota",Boathouse Brewpub & Restaurant,25
14379,17208,Iceland,Ölvisholt Brugghús,36
8805,44952,"United States, California",Midnight Jack Brewing Company,10


In [48]:
rb_breweries.sample(4)

,id,location,name,nbr_beers
7469,15988,Australia,115 Grill & Brewhouse,2
6358,28313,Switzerland,Mükon,7
6058,23198,Switzerland,1386,2
22849,14626,Belgium,Het Brouwersatelier (was: De Relatiebrouwer),12


The columns are explicit and are the same for the 2 websites. Each row is a brewery.

**Rating Dataframes**

In [49]:
ba_ratings.sample(4)

,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating
4088767,Michelob Ultra Fruit Pomegranate Raspberry,36327.0,Anheuser-Busch,29.0,Light Lager,4.20,2014-06-11 10:00:00,Cotton_27,cotton_27.719747,NaN,NaN,NaN,NaN,NaN,1.50
703429,Krombacher Radler,13070.0,Krombacher Brauerei,566.0,Fruit / Vegetable Beer,2.50,2015-07-04 10:00:00,KrisDLSmith,krisdlsmith.682689,3.50,3.25,3.50,3.50,3.50,3.44
4444969,Stoudt's Double IPA (India Pale Ale),20470.0,Stoudts Brewing Co.,394.0,American Double / Imperial IPA,9.43,2015-04-11 10:00:00,Lettsgo3,lettsgo3.807337,4.50,4.25,4.00,4.25,4.25,4.24
6413255,Great Lakes Conway's Irish Ale,5096.0,Great Lakes Brewing Co.,73.0,Irish Red Ale,6.50,2013-02-24 11:00:00,wedge,wedge.4389,3.75,3.50,4.25,4.25,4.00,3.99


In [50]:
rb_ratings.sample(4)

,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating
6169064,Youngs Oatmeal Stout (London Stout),22412.0,Charles Wells &#40;Marstons&#41;,754.0,Stout,5.2,2003-03-16 11:00:00,muzzlehatch,956.0,5.0,7.0,4.0,8.0,16.0,4.0
4979244,Hoppin Frog Frogs Hollow Double Pumpkin Ale,108891.0,Hoppin Frog Brewery,8035.0,Spice/Herb/Vegetable,8.4,2009-10-24 10:00:00,MsLucky13,96827.0,5.0,10.0,5.0,10.0,20.0,5.0
1782810,Nova Paka Granát,4854.0,Pivovar Nová Paka,835.0,Polotmavý,5.0,2005-09-17 10:00:00,pilsnerrogge,2862.0,3.0,2.0,1.0,3.0,8.0,1.7
4801265,Free State Old Backus Barleywine,2943.0,Free State Brewing Company,510.0,Barley Wine,10.5,2005-12-09 11:00:00,JCW,5655.0,4.0,8.0,4.0,7.0,15.0,3.8


The columns are the same for the 2 Dataframes. Each row corresponds to an individual review. Most column names are explicit. 
- 'appearance','aroma', 'palate','taste' correspond to aspect ratings (out of 5)
- 'overall' is the mean of the 4 aspect ratings
- 'rating' is the final rating given by the user to the beer

# 0) Data cleaning

In [51]:
# remove useless columns (done)
# make sure each column has the right type (done)
# deal with missing or Nan values (partially done)
# check the correspondance between brewery_id in the beers DataFrames and brewery_id in the breweries Dataframes
# set all US locations to 'United States' (remove state information) (done)
# remove any embedded HTML links in the location strings (done)
# remove countries with too few reviewers (done)

## Filtering Dataframes

Let us start by removing columns in the different Dataframes that we will not use in our analysis.

The following rows will not be used in our analysis:
nbr_reviews, ba_score, bros_score, abv, avg_computed, zscore, nbr_matched_valid_ratings and avg_matched_valid_ratings, overall_score and style_score.

Let us remove them.

In [52]:
useless_columns_ba = ['nbr_reviews', 'ba_score', 'bros_score', 'abv', 'avg_computed', 'zscore', 'nbr_matched_valid_ratings', 'avg_matched_valid_ratings']
ba_beers = ba_beers.drop(columns=useless_columns_ba)
print(ba_beers.columns)

Index(['beer_id', 'beer_name', 'brewery_id', 'brewery_name', 'style',
       'nbr_ratings', 'avg'],
      dtype='object')


In [53]:
useless_columns_rb = [col for col in useless_columns_ba if col not in ['nbr_reviews','ba_score', 'bros_score']] + ['overall_score', 'style_score']
rb_beers = rb_beers.drop(columns=useless_columns_rb)
print(rb_beers.columns)

Index(['beer_id', 'beer_name', 'brewery_id', 'brewery_name', 'style',
       'nbr_ratings', 'avg'],
      dtype='object')


We will also not use the timestamps indicating the time when users joined the platforms, so let us remove this as well.

In [54]:
ba_users = ba_users.drop(columns='joined')
rb_users = rb_users.drop(columns='joined')
print(ba_users.columns)

Index(['nbr_ratings', 'nbr_reviews', 'user_id', 'user_name', 'location'], dtype='object')


## Verifying value types

Let us verify that the values in the different columns of the different Dataframes have the appropriate type.

In [55]:
print(ba_beers.dtypes,'\n','\n',rb_beers.dtypes)

beer_id           int64
beer_name        object
brewery_id        int64
brewery_name     object
style            object
nbr_ratings       int64
avg             float64
dtype: object 
 
 beer_id           int64
beer_name        object
brewery_id        int64
brewery_name     object
style            object
nbr_ratings       int64
avg             float64
dtype: object


In [56]:
print(ba_users.dtypes,'\n','\n',rb_users.dtypes)

nbr_ratings     int64
nbr_reviews     int64
user_id        object
user_name      object
location       object
dtype: object 
 
 nbr_ratings     int64
user_id         int64
user_name      object
location       object
dtype: object


In [57]:
print(ba_breweries.dtypes,'\n','\n',rb_breweries.dtypes)

id            int64
location     object
name         object
nbr_beers     int64
dtype: object 
 
 id            int64
location     object
name         object
nbr_beers     int64
dtype: object


In [58]:
columns_to_convert = ['beer_name', 'brewery_name', 'style']

ba_beers[columns_to_convert] = ba_beers[columns_to_convert].apply(lambda col: col.astype(str))
rb_beers[columns_to_convert] = rb_beers[columns_to_convert].apply(lambda col: col.astype(str))
print(ba_beers.dtypes,'\n','\n',rb_beers.dtypes)

beer_id           int64
beer_name        object
brewery_id        int64
brewery_name     object
style            object
nbr_ratings       int64
avg             float64
dtype: object 
 
 beer_id           int64
beer_name        object
brewery_id        int64
brewery_name     object
style            object
nbr_ratings       int64
avg             float64
dtype: object


In [59]:
print(ba_ratings.dtypes,'\n','\n',rb_ratings.dtypes)

beer_name               object
beer_id                float64
brewery_name            object
brewery_id             float64
style                   object
abv                    float64
date            datetime64[ns]
user_name               object
user_id                 object
appearance             float64
aroma                  float64
palate                 float64
taste                  float64
overall                float64
rating                 float64
dtype: object 
 
 beer_name               object
beer_id                float64
brewery_name            object
brewery_id             float64
style                   object
abv                    float64
date            datetime64[ns]
user_name               object
user_id                float64
appearance             float64
aroma                  float64
palate                 float64
taste                  float64
overall                float64
rating                 float64
dtype: object


The types of the values in the different columns of the different Dataframes seem appropriate.

## Dealing with missing values

In [82]:
ba_beers['avg'].value_counts()

avg
4.00    7783
3.75    7059
3.50    5946
3.88    3307
4.25    2871
        ... 
1.14       1
1.19       1
1.04       1
1.05       1
1.27       1
Name: count, Length: 401, dtype: int64

In [83]:
ba_beers

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,avg
0,166064,Nashe Moskovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,NaN
1,166065,Nashe Pivovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,NaN
2,166066,Nashe Shakhterskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,NaN
3,166067,Nashe Zhigulevskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,NaN
4,166063,Zhivoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,NaN
...,...,...,...,...,...,...,...
280818,19139,Kölsch Ale,885,Summit Station Restaurant & Brewery,Kölsch,3,2.71
280819,19140,Nut Brown Ale,885,Summit Station Restaurant & Brewery,English Brown Ale,2,3.10
280820,19146,Octoberfest,885,Summit Station Restaurant & Brewery,Märzen / Oktoberfest,0,NaN
280821,2805,Scotch Ale,885,Summit Station Restaurant & Brewery,Scotch Ale / Wee Heavy,0,NaN


In [84]:
ba_beers_ = ba_beers[~pd.isna(ba_beers['avg'])].reset_index() # avg = NaN valued beers are removed since there are not any reviews
ba_beers_

"""
# We are checking if there are beers with 0 averages and remove them:
rows_with_zero_avg = ba_beers_[ba_beers_['avg'] == 0]
#display(rows_with_zero_avg)
ba_beers_ = ba_beers_[ba_beers_['avg'] != 0]
ba_beers_ = ba_beers_.reset_index(drop=True)
# We are checking if there are  beers still left with 0 averages and remove them:
rows_with_zero_ratings=ba_beers_[ba_beers_['nbr_ratings'] == 0]
#display(rows_with_zero_ratings)
ba_beers_cleaned = ba_beers_[ba_beers_['nbr_ratings'] != 0]
ba_beers_cleaned = ba_beers_cleaned.reset_index(drop=True)
#ba_beers_cleaned is the dataset with both columns avg and nbr_ratings purified from the NaN and 0 values-
"""

"\n# We are checking if there are beers with 0 averages and remove them:\nrows_with_zero_avg = ba_beers_[ba_beers_['avg'] == 0]\n#display(rows_with_zero_avg)\nba_beers_ = ba_beers_[ba_beers_['avg'] != 0]\nba_beers_ = ba_beers_.reset_index(drop=True)\n# We are checking if there are  beers still left with 0 averages and remove them:\nrows_with_zero_ratings=ba_beers_[ba_beers_['nbr_ratings'] == 0]\n#display(rows_with_zero_ratings)\nba_beers_cleaned = ba_beers_[ba_beers_['nbr_ratings'] != 0]\nba_beers_cleaned = ba_beers_cleaned.reset_index(drop=True)\n#ba_beers_cleaned is the dataset with both columns avg and nbr_ratings purified from the NaN and 0 values-\n"

In [85]:
rb_beers_ = rb_beers[~pd.isna(rb_beers['avg'])].reset_index(drop=True) # avg = NaN valued beers are removed since there are not any reviews
rb_beers_
"""
# We are checking if there are beer with 0 ratings and remove them:
#rows_with_zero_ratings_rb=rb_beers_[rb_beers_['nbr_ratings'] == 0]
#display(rows_with_zero_ratings_rb)
rb_beers_ = rb_beers_[rb_beers_['nbr_ratings'] != 0].reset_index(drop=True)

# We are checking if there are beers still left with 0 averages and remove them:
#rows_with_zero_avg_rb = rb_beers_[rb_beers_['avg'] == 0]
#display(rows_with_zero_avg_rb)
rb_beers_cleaned = rb_beers_[rb_beers_['avg'] != 0].reset_index(drop=True)

display(rb_beers_cleaned)

#rb_beers_cleaned is the dataset with both columns avg and nbr_ratings purified from the NaN and 0 values-
"""

"\n# We are checking if there are beer with 0 ratings and remove them:\n#rows_with_zero_ratings_rb=rb_beers_[rb_beers_['nbr_ratings'] == 0]\n#display(rows_with_zero_ratings_rb)\nrb_beers_ = rb_beers_[rb_beers_['nbr_ratings'] != 0].reset_index(drop=True)\n\n# We are checking if there are beers still left with 0 averages and remove them:\n#rows_with_zero_avg_rb = rb_beers_[rb_beers_['avg'] == 0]\n#display(rows_with_zero_avg_rb)\nrb_beers_cleaned = rb_beers_[rb_beers_['avg'] != 0].reset_index(drop=True)\n\ndisplay(rb_beers_cleaned)\n\n#rb_beers_cleaned is the dataset with both columns avg and nbr_ratings purified from the NaN and 0 values-\n"

## Checking the correspondance between brewery_id in the beers DataFrames

## Removing state information

In [28]:
import warnings
warnings.filterwarnings('ignore')

In [47]:
def edit_location(data_name):
    data_name_c = data_name.copy()
    for i in range(len(data_name['location'])):
        if len(data_name['location'][i]) > 10:
            if 'United States' in data_name['location'][i]: # Remove state names
                data_name_c['location'][i] = 'United States'
            elif ',' in data_name['location'][i]:
                data_name_c['location'][i] = data_name['location'][i][:(data_name['location'][i].index(','))] # Removing for the double names ( such as 'United Kingdom,England' )
            elif 'href' in data_name['location'][i]:
                data_name_c.drop(i)
    return data_name_c

In [30]:
ba_breweries = edit_location(ba_breweries)

In [31]:
ba_breweries['location']

0           Kyrgyzstan
1           Kyrgyzstan
2           Kyrgyzstan
3           Kyrgyzstan
4           Kyrgyzstan
             ...      
16753          Germany
16754            Aruba
16755    United States
16756    United States
16757    United States
Name: location, Length: 16758, dtype: object

In [32]:
rb_breweries = edit_location(rb_breweries)

In [33]:
rb_breweries['location']

0                   Gabon
1        Northern Ireland
2        Northern Ireland
3        Northern Ireland
4        Northern Ireland
               ...       
24184             Albania
24185             Albania
24186             Albania
24187             Albania
24188             Albania
Name: location, Length: 24189, dtype: object

## Removing HTML links

In [ ]:
# Done above

## Removing the countries who have too few reviewers

In [61]:
ba_users = ba_users[~pd.isna(ba_users['location'])].reset_index() # location = NaN valued users are removed
rb_users = rb_users[~pd.isna(rb_users['location'])].reset_index() # location = NaN valued users are removed
ba_users

,index,nbr_ratings,nbr_reviews,user_id,user_name,location
0,0,7820,465,nmann08.184925,nmann08,"United States, Washington"
1,1,2521,2504,stjamesgate.163714,StJamesGate,"United States, New York"
2,2,1797,1143,mdagnew.19527,mdagnew,Northern Ireland
3,3,31,31,helloloser12345.10867,helloloser12345,Northern Ireland
4,4,604,604,cypressbob.3708,cypressbob,Northern Ireland
...,...,...,...,...,...,...
122420,153698,1,0,eturchick.374415,ETurchick,"United States, California"
122421,153699,1,1,everman.532342,Everman,"United States, California"
122422,153700,1,1,justin0001.352175,Justin0001,"United States, California"
122423,153702,1,1,joetex.800347,JoeTex,"United States, California"


In [62]:
rb_users

,index,nbr_ratings,user_id,user_name,location
0,0,1890,175852,Manslow,Poland
1,1,89,442761,MAGICuenca91,Spain
2,2,169,288889,Sibarh,Poland
3,3,3371,250510,fombe89,Spain
4,4,13043,122778,kevnic2008,Germany
...,...,...,...,...,...
50587,70167,1,181614,HaraldS,Norway
50588,70169,1,134893,stamfordbus,England
50589,70170,1,327816,fobia405,Belgium
50590,70172,3,82020,klesidra,Slovenia


In [64]:
ba_users_ = edit_location(ba_users)
rb_users_ = edit_location(rb_users)

In [71]:
count = rb_users_['location'].value_counts() < 20
count[count == True]

location
Panama                   True
Moldova                  True
Costa Rica               True
Ecuador                  True
Virgin Islands (U.S.)    True
                         ... 
Honduras                 True
Falkland Islands         True
Kyrgyzstan               True
Papua New Guinea         True
Tibet                    True
Name: count, Length: 112, dtype: bool

In [72]:
rb_users_c = rb_users_.copy()
for i in range(len(rb_users_['location'])):
    if rb_users_['location'][i] in count :
        rb_users_c.drop(i)

KeyboardInterrupt: 

# 1) Link between culture and taste

## a) Beer style preferences

In [ ]:
# use clustering techniques to determine beer style is most popular in each country / geographic area
# use time information to determine if regional beer style preferences are stable (which would suggest that they are 
# strongly affected by culture)or if they vary over time

## b) Importance of specific beer attributes

In [ ]:
# perform linear regression between attribute ratings the final rating for all countries together and compare coefficients for each attribute
# perform linear regression between attribute ratings the final rating for the different countries separately and observe the distribution of the coefficients for the different attributes across countries

# 2) Location-related biases in ratings

## a) Cultural biases

In [ ]:
# determine the final rating for each country/ geographic area
# determine if the final rating for each country/ geographic area is the same using statistical tests

## b) Beer origin bias

In [ ]:
# compare the final rating of domestic vs foreign beers and determine if there is a significant difference using statistical tests
# determine if the final rating of a given beer is correlated with the number of reviewers from the country where the beer comes from who reviewed that beer (scatter plot + Pearson’s correlation coefficient + regression)
# isolate beer enthusiasts (who wrote a very large number of reviews) and compare the final rating of domestic vs foreign beers and determine if there is a significant difference using statistical tests

# 3) Other biases

## a) Seasonal biases

In [ ]:
# use the time information to determine the season during which each rating was posted (only consider countries with 4 seasons)
# group ratings by season
# within each group, determine the average final rating of each beer style
# compare the results for the different seasons

## b) Experience biais

In [ ]:
# isolate users who gave a lot of ratings (based on a chosen threshold)
# for each user, sort their reviews chronologically and assign an "experience level" (predefined values that will be the same for all users: n<o<p) to each rating based the count of reviews posted by that user up to that rating: new reviewer (for the first n reviews), amateur (for the n+1 th review up to the oth review), expert (for the o+1 th review until the last review)
# calculate the average final rating for each experience level across all users
# represent results as a bar plot
# if a particular trend is visible,perform a paired t-test (for early vs. late reviews by the same user) to test if the rating decrease or increase is statistically significant